In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import time
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import math

import warnings
#warnings.filterwarnings("ignore")




start_time = time.time()

df_data=pd.read_csv("../input/jane-street-market-prediction/train.csv")
df_data=df_data[(df_data.weight>0)]

df_data['action2']=np.where((df_data['resp']*df_data['weight'])>0,1,0)

df_data.fillna(df_data.mean(),inplace=True)

num_dev_samples=int(len(df_data)*0.8)
#print(num_dev_samples, len(df_data))

df_dev_data=df_data[:num_dev_samples]
df_val_data=df_data[num_dev_samples:]

#print('len of df_dev_data ', len(df_dev_data))
#print('len of df_val_data ', len(df_val_data))
#print(df_data[['date','action2']].groupby(['date']).agg(['mean','count']))

features=df_data.loc[:,df_data.columns.str.contains('feature')].columns.values

np_dev_label= np.array(df_dev_data[['action2']].values, dtype="uint8")

np_val_label= np.array(df_val_data[['action2']].values, dtype="uint8")

np_dev_features=df_dev_data.loc[:, features].values
np_val_features=df_val_data.loc[:, features].values

np_dev_features_mean=np.mean(np_dev_features, axis=0)

df_val_othercols=df_val_data[['resp', 'weight','date']]

sc = StandardScaler()
np_dev_features = sc.fit_transform(np_dev_features)
np_val_features = sc.transform(np_val_features)


def utility(arr_utility, iteration):
    
    sum_date=0
    sum_datesq=0
    
    for i in range(0,len(arr_utility)):
        sum_date = sum_date + arr_utility[i]
        sum_datesq = sum_datesq + (arr_utility[i]*arr_utility[i])
    
    t=(sum_date/math.sqrt(sum_datesq)) * (math.sqrt(250/len(arr_utility)))
    
    u=min(max(t,0),6)*(sum_date)
    #utility_output[x] = u '
    
    print('utility at iteration ', iteration, '  is ',  u, '\n')

    return 1 




def create_nn(num_hiddenunits):
    
    #print('Model running with hidden units as ', num_hiddenunits)
    
    inp=tf.keras.layers.Input(shape=(130,))
    x=tf.keras.layers.Dense(num_hiddenunits, activation="relu")(inp)  
    out=tf.keras.layers.Dense(1, activation="softsign" )(x)
    
    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss='binary_crossentropy',
        metrics='binary_accuracy',
    )
    
    
    return model



hidden_units=[100]

for i in range(len(hidden_units)):
    clf = create_nn(hidden_units[i])
    clf.fit(np_dev_features, np_dev_label, epochs=20, batch_size=5000, validation_data=(np_val_features, np_val_label))
    #print('\n')
    #print('\n')
    #Get utility
    '''
    val_predctn=clf.predict(np_val_features)
    val_predctn_sum=np.sum(val_predctn, axis=1)
    df_val_othercols['pred']=val_predctn_sum.tolist()
    df_val_othercols['action']=np.where(df_val_othercols['pred']>=0.5,1,0)
    df_val_othercols['prod']=df_val_othercols['weight']*df_val_othercols['action']*df_val_othercols['resp']
    df_utility = df_val_othercols[['date','prod']].groupby(['date']).sum()
    arr_utility = np.array(df_utility['prod'].values)
    k=utility(arr_utility, hidden_units[i])
    '''

    
import janestreet
env = janestreet.make_env()
iter_test = env.iter_test() 
#i=0

for (test_df, sample_prediction_df) in iter_test:
    

    if test_df['weight'].item() > 0:
        #i+=1
        x_tt = test_df.loc[:, features].values

        if np.isnan(x_tt.sum()):
            x_tt= np.isnan(x_tt)*np_dev_features_mean + np.nan_to_num(x_tt)
        #x_tt=(x_tt-np_dev_features_mean)/np_dev_features_std
        #x_tt -= mean
        #x_tt /= std
        
        x_tt = sc.transform(x_tt)


        pred=clf(x_tt, training = False).numpy()
        sample_prediction_df.action = np.where(pred[0][0] >= 0.5, 1, 0).astype(int)
        '''
        if i%50 == 0:
            print('i is ', i)
            print(x_tt[0][1], x_tt[0][2], x_tt[0][3] )
            print(pred)
        '''
    else:
        
        sample_prediction_df.action = 0
        
    #i=i+1
    env.predict(sample_prediction_df)


